# Imports

In [1]:
from neo4j import GraphDatabase

# Establish Connection

In [2]:
URI = "neo4j+s://d4757d6d.databases.neo4j.io"
AUTH = ("neo4j", "dmmaXVPJMAgniJ3aQW50hGhu5YjaAOdR-nVIB7n2H8s")

driver =  GraphDatabase.driver(URI, auth=AUTH) 
driver.verify_connectivity()

In [3]:
def read_cypher_query_from_file(file_path):
    with open(file_path, "r") as file:
        query = file.read()
    return query

In [21]:
def run_cypher_query(query):
    with driver.session() as session:
        result = session.run(query)
        return list(result)

In [5]:
query_1 = read_cypher_query_from_file("moviesDB.cypher")

# Create Database

In [6]:
run_cypher_query(query_1)

# Queries

## 1 Find all actors who have also directed movies

In [7]:
query = """
MATCH (p:Person)-[:ACTED_IN]->(), (p)-[:DIRECTED]->()
RETURN p.name
"""

In [22]:
results = run_cypher_query(query)

In [40]:
actors=set()
for record in results:
    actors.update(set(record))

In [41]:
actors

{'Clint Eastwood',
 'Danny DeVito',
 'James Marshall',
 'Tom Hanks',
 'Werner Herzog'}

## 2 List all movies released after a certain year, ordered by their release date

In [62]:
year = 2015
query = f"""
MATCH (m:Movie)
WHERE m.released > {year}
RETURN m.title, m.released
ORDER BY m.released ASC
"""

In [63]:
results = run_cypher_query(query)

In [64]:
movies=set()
for record in results:
    movies.update(set(record))

In [65]:
movies

{2019, 'Joker', 'Parasite'}

## 3 List all movies and the roles a Tom Hanks has played in them

In [78]:
query = """
MATCH (p:Person {name: 'Tom Hanks'})-[r:ACTED_IN]->(m:Movie)
RETURN m.title, r.roles
"""

In [79]:
results = run_cypher_query(query)

In [81]:
results

[<Record m.title="You've Got Mail" r.roles=['Joe Fox']>,
 <Record m.title='Sleepless in Seattle' r.roles=['Sam Baldwin']>,
 <Record m.title='Joe Versus the Volcano' r.roles=['Joe Banks']>,
 <Record m.title='That Thing You Do' r.roles=['Mr. White']>,
 <Record m.title='Cloud Atlas' r.roles=['Zachry', 'Dr. Henry Goose', 'Isaac Sachs', 'Dermot Hoggins']>,
 <Record m.title='The Da Vinci Code' r.roles=['Dr. Robert Langdon']>,
 <Record m.title='The Green Mile' r.roles=['Paul Edgecomb']>,
 <Record m.title='Apollo 13' r.roles=['Jim Lovell']>,
 <Record m.title='Cast Away' r.roles=['Chuck Noland']>,
 <Record m.title="Charlie Wilson's War" r.roles=['Rep. Charlie Wilson']>,
 <Record m.title='The Polar Express' r.roles=['Hero Boy', 'Father', 'Conductor', 'Hobo', 'Scrooge', 'Santa Claus']>,
 <Record m.title='A League of Their Own' r.roles=['Jimmy Dugan']>]

## 4 Find the top 5 most reviewed movies and their review counts

In [89]:
query = """
MATCH (m:Movie)<-[:REVIEWED]-()
RETURN m.title, COUNT(*) AS reviewCount
ORDER BY reviewCount DESC
LIMIT 5
"""

In [90]:
results = run_cypher_query(query)

In [91]:
results

[<Record m.title='The Replacements' reviewCount=3>,
 <Record m.title='The Da Vinci Code' reviewCount=2>,
 <Record m.title='The Birdcage' reviewCount=1>,
 <Record m.title='Unforgiven' reviewCount=1>,
 <Record m.title='Cloud Atlas' reviewCount=1>]